In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, first

# 1. Iniciando a Sessão Spark
# SparkSession: É o ponto de entrada. Sem ele, nada acontece.
# .master(...): Diz quem manda. 'spark://spark-master:7077' é o endereço do container mestre.
# .config(...defaultFS...): Ensina o Spark que, quando eu falar "/pasta", é no HDFS, não no local.
# .config(...hive...): Habilita suporte a salvar tabelas SQL.
spark = SparkSession.builder \
    .appName("ETL_Silver") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

def carga_gold():
    print("Iniciando Camada Gold...")
    
    # 1. Leitura Otimizada
    # Lemos Parquet, não JSON. É muito mais rápido.
    df_suicidio = spark.read.parquet("/datalake/silver/raw_suicidio")
    df_depressao = spark.read.parquet("/datalake/silver/raw_depressao")
    
    # 2. Join (Cruzamento)
    # on=["Pais", "Ano"]: A chave da junção.
    # how="inner": Só mantém países que tenham dados AMBOS de suicídio e depressão.
    df_gold = df_suicidio.join(df_depressao, on=["Pais", "Ano"], how="inner")
    
    # 3. Salvar Tabela Hive (Para o Superset)
    # saveAsTable: Registra os metadados no Hive Metastore.
    # Isso permite fazer SQL: "SELECT * FROM saude_mental_analitica" no Trino/Superset.
    df_gold.write.mode("overwrite").saveAsTable("saude_mental_analitica")
    print("✅ Tabela Hive 'saude_mental_analitica' disponível para SQL!")

    # 4. Salvar CSV (Para Download/Power BI)
    # coalesce(1): O Spark divide arquivos (sharding). Isso força ele a juntar tudo em 1 arquivo só.
    # option("header", "true"): Escreve o cabeçalho no CSV.
    df_gold.coalesce(1).write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv("/datalake/gold/analise_final_csv")
        
    print("✅ CSV gerado em /datalake/gold/analise_final_csv")

    # Comando de sistema (Shell) para pegar o arquivo do HDFS
    # getmerge: Pega a pasta do HDFS (/datalake/...) e junta num arquivo local
    # O destino deve ser a pasta que está sincronizada com seu Windows (/home/jovyan/data)
    exit_code = os.system("hdfs dfs -getmerge /datalake/gold/analise_final_csv /home/jovyan/notebooks/datasets/resultado_powerbi.csv")

    if exit_code == 0:
        print("✅ Arquivo 'resultado_powerbi.csv' gerado com sucesso na sua pasta local!")
    else:
        print("❌ Erro ao exportar o arquivo.")
    
# Rodando a finalização
carga_gold()
spark.stop()
print("Sessão Spark finalizada.")